pip install pandas matplotlib sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
!jq . "COVID-19/dati-json/dpc-covid19-ita-province.json" > "dpc-covid19-ita-province-nobom.json"
!jq . "COVID-19/dati-json/dpc-covid19-ita-regioni.json" > "dpc-covid19-ita-regioni-nobom.json"
!jq . "COVID-19/dati-json/dpc-covid19-ita-andamento-nazionale.json" > "dpc-covid19-ita-andamento-nazionale-nobom.json"

In [ ]:
ita_province = pd.read_json('dpc-covid19-ita-province-nobom.json')
ita_regioni = pd.read_json('dpc-covid19-ita-regioni-nobom.json')
ita_andamento_nazionale = pd.read_json('dpc-covid19-ita-andamento-nazionale-nobom.json')

In [ ]:
data = ita_regioni.set_index(['denominazione_regione', 'data']).sort_index().reset_index()

data['data'] = pd.to_datetime(data['data'])
# del data['long'], data['lat'], data['stato'], data['codice_regione'], data['nuovi_attualmente_positivi']
# del data['totale_attualmente_positivi'], data['totale_ospedalizzati']

In [ ]:
def carica_province():
    x = ita_province[['data', 'denominazione_provincia', 'totale_casi']].copy().sort_values(['denominazione_provincia', 'data'])
    x['data'] = pd.to_datetime(x['data'])
    x['luogo'] = x['denominazione_provincia']
    del x['denominazione_provincia']
    x['nuovi_casi'] = x.groupby('luogo').totale_casi.diff()
    print(x.luogo)
    x['stima_attualmente_positivi'] = x.groupby('luogo').totale_casi.shift()
    x['crescita'] = x['nuovi_casi'] / x['stima_attualmente_positivi']

    x = x[x['luogo'] != 'In fase di definizione/aggiornamento']
    
    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    
    return x

In [ ]:
def carica_regioni():
    x = ita_regioni[['data', 'denominazione_regione', 'totale_casi', 'totale_positivi']].copy().sort_values(['denominazione_regione', 'data'])
    x['data'] = pd.to_datetime(x['data'])
    x['luogo'] = x['denominazione_regione']
    del x['denominazione_regione']
    x['nuovi_casi'] = x.groupby('luogo').totale_casi.diff()
    x['stima_attualmente_positivi'] = x.groupby('luogo').totale_positivi.shift()
    x['crescita'] = x['nuovi_casi'] / x['stima_attualmente_positivi']

    x = x[x['luogo'] != 'In fase di definizione/aggiornamento']
    
    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    
    return x

In [ ]:
def carica_andamento_nazionale():
    x = ita_andamento_nazionale[[
        'data', 'totale_casi', 'totale_positivi', 'tamponi', 'dimessi_guariti', 'deceduti', 'ricoverati_con_sintomi', 'terapia_intensiva', 'isolamento_domiciliare'
    ]].copy().sort_values('data')
    x['data'] = pd.to_datetime(x['data'])
    #x['giorno'] = x.groupby('data').ngroup()
    x['nuovi_casi'] = x.totale_casi.diff()
    x['stima_attualmente_positivi'] = x.totale_positivi.shift()
    x['crescita'] = x['nuovi_casi'] / x['stima_attualmente_positivi']
    x['luogo'] = 'Italia'

    x = x.replace([np.inf, -np.inf], np.nan).dropna()
    
    return x

In [ ]:
dati_province = carica_province()
dati_regioni = carica_regioni()
dati_andamento_nazionale = carica_andamento_nazionale()

In [ ]:
x = dati_andamento_nazionale.set_index('data')
x[[
    'deceduti',
    'terapia_intensiva',
    'ricoverati_con_sintomi',
    'isolamento_domiciliare',
    'dimessi_guariti',
]].diff().plot.bar(figsize=(20, 10), stacked=True)

In [ ]:
dati = dati_province
dati = dati[dati['data'] > '2020-03-10']

dati.plot.scatter(
    'data', 'crescita', 
    s=0.03 * dati['totale_casi'], 
    alpha=0.3,
    logy=True,
    ylim=(0.01, 1),
    figsize=(10, 10),
)

dati = dati_andamento_nazionale
dati = dati[dati['data'] > '2020-03-10']


dati.set_index('data').crescita.plot(
#     'data', 'crescita', 
#     s=0.1 * dati['totale_casi'], 
    alpha=1,
    logy=True,
    ylim=(0.01, 1),
    figsize=(10, 10),
    style='rx',
    markersize=20,
    markeredgewidth=2,
)


In [ ]:
dati = dati_andamento_nazionale.set_index('data')[['tamponi', 'totale_casi']].diff()
dati = dati[-10:]
dati.plot(figsize=(5, 5), style='x-')
(dati.totale_casi / dati.tamponi).rename('hit_rate').plot(secondary_y=True, legend=True)

In [ ]:
xlim_date = [pd.to_datetime('2020-02-26'), pd.to_datetime('2020-04-07')]
xlim = [d.to_datetime64() for d in xlim_date]

for risoluzione_nome, dati, scatter_size, min_casi in [
    ('province', dati_province, 0.1, 2000),
    ('regioni', dati_regioni, 0.05, 4000),
]:
    for logy in [True, False]:
        if logy:
            ylim = (0.005, 5)
        else:
            ylim = (-0.1, 1.5)

        italia = dati_andamento_nazionale

        italia.plot.scatter(
            'data', 
            'crescita', 
            s=0.05 * italia['totale_casi'], 
            color='r', 
            alpha=0.2,
            logy=logy, 
            ylim=ylim, 
            figsize=(20, 15),
        )
        ax = plt.gca()

        italia.set_index('data')['crescita'].plot(style='o-r', alpha=0.5, linewidth=2)

        dati.plot.scatter(
            'data', 'crescita', 
            s=scatter_size*dati['totale_casi'], 
            alpha=0.15,
            ax=ax, 
        )

        for coeff, style in [
            [1.00, 'r'],
            [0.50, 'r'],
            [0.20, 'r'],
            [0.10, 'y'],
            [0.05, 'y'],
            [0.02, 'g'],
        ]:
            ax.plot(xlim, [coeff, coeff], style, linewidth=1, alpha=0.3)
            ax.annotate(f"+{coeff * 100:.0f}%", (xlim[1], coeff * 1.1), alpha=0.5, ha='right')

        train = italia[['data', 'crescita', 'totale_casi']].copy()

        def make_predictor(data):
            return pd.DataFrame(dict(
                secondi=(data - xlim_date[0]).dt.total_seconds(),
                sin_giorno=((data - xlim_date[0]).dt.total_seconds() * 2 * np.pi / (60 * 60 * 24 * 7)).apply(np.sin),
                cos_giorno=((data - xlim_date[0]).dt.total_seconds() * 2 * np.pi / (60 * 60 * 24 * 7)).apply(np.cos),
            ))
        
        dr = pd.date_range(*xlim_date, freq='6h').to_series()

        for weighted in [True]:
            regr = LinearRegression()
            if weighted:
                regr.fit(
                    make_predictor(train['data']).values, 
                    train['crescita'].values.reshape(-1, 1), 
                    train['totale_casi'].values,
                )
            else:
                regr.fit(
                    make_predictor(train['data']).values, 
                    train['crescita'].values.reshape(-1, 1), 
                )

            pred_crescita = pd.Series(
                regr.predict(make_predictor(dr).values)[:,0],
                index=dr,
            )
            
            if weighted:
                pred_crescita.plot(c='r', linewidth=4, alpha=0.3)
            else:
                pred_crescita.plot(c='b', linewidth=3, alpha=0.3)

        for luogo, d in dati.groupby('luogo'):
            if not d['totale_casi'].iloc[-1] > min_casi:
                continue

            d = d.copy().set_index('data', drop=False)
            d['crescita'].where(d['crescita'] > 0).plot(
                xlim=xlim,
                ylim=ylim,
                alpha=0.2,
                linewidth=3,
        #         c='k',
            )

            x = d
            x = x[x['data'] > xlim[0]]
            #x = x[x['totale_casi'] > 300]
            x = x[ylim[0] < x['crescita']]
            x = x[x['crescita'] < ylim[1]]
            #x = x[(x['crescita'] == x['crescita'].max()) | (x['crescita'] == x['crescita'].min())]

            for y in x.iloc:
                ax.annotate(luogo, (y['data'].to_datetime64(), y['crescita']), c='k', va='center', ha='center', alpha=0.5)

        for y in italia.iloc[::5].iloc:
            ax.annotate('ITALIA', (y['data'].to_datetime64(), y['crescita']), c='r', va='bottom', ha='center', alpha=0.5, fontsize=14)

        if logy:
            logy_name = 'log'
        else:
            logy_name = 'lin'
        plt.savefig(f"{risoluzione_nome}-{logy_name}-{dati['data'].max().to_pydatetime().date().isoformat()}.png")

In [ ]:
xlim_date = [pd.to_datetime('2020-02-26'), pd.to_datetime('2020-04-07')]
xlim = [d.to_datetime64() for d in xlim_date]

for risoluzione_nome, dati, scatter_size, min_casi in [
    ('province', dati_province, 0.1, 2000),
    ('regioni', dati_regioni, 0.05, 4000),
]:
    for logy in [True, False]:
        if logy:
            ylim = (0.005, 5)
        else:
            ylim = (-0.1, 1.5)

        italia = dati_andamento_nazionale

        italia.plot.scatter(
            'data', 
            'crescita', 
            s=0.05 * italia['totale_casi'], 
            color='r', 
            alpha=0.2,
            logy=logy, 
            ylim=ylim, 
            figsize=(20, 15),
        )
        ax = plt.gca()

        italia.set_index('data')['crescita'].plot(style='o-r', alpha=0.5, linewidth=2)

        dati.plot.scatter(
            'data', 'crescita', 
            s=scatter_size*dati['totale_casi'], 
            alpha=0.15,
            ax=ax, 
        )

        for coeff, style in [
            [1.00, 'r'],
            [0.50, 'r'],
            [0.20, 'r'],
            [0.10, 'y'],
            [0.05, 'y'],
            [0.02, 'g'],
        ]:
            ax.plot(xlim, [coeff, coeff], style, linewidth=1, alpha=0.3)
            ax.annotate(f"+{coeff * 100:.0f}%", (xlim[1], coeff * 1.1), alpha=0.5, ha='right')

        train = italia[['data', 'crescita', 'totale_casi']].copy()

        def make_predictor(data):
            return pd.DataFrame(dict(
                secondi=(data - xlim_date[0]).dt.total_seconds(),
                sin_giorno=((data - xlim_date[0]).dt.total_seconds() * 2 * np.pi / (60 * 60 * 24 * 7)).apply(np.sin),
                cos_giorno=((data - xlim_date[0]).dt.total_seconds() * 2 * np.pi / (60 * 60 * 24 * 7)).apply(np.cos),
            ))
        
        dr = pd.date_range(*xlim_date, freq='6h').to_series()

        for weighted in [True]:
            regr = LinearRegression()
            if weighted:
                regr.fit(
                    make_predictor(train['data']).values, 
                    train['crescita'].values.reshape(-1, 1), 
                    train['totale_casi'].values,
                )
            else:
                regr.fit(
                    make_predictor(train['data']).values, 
                    train['crescita'].values.reshape(-1, 1), 
                )

            pred_crescita = pd.Series(
                regr.predict(make_predictor(dr).values)[:,0],
                index=dr,
            )
            
            if weighted:
                pred_crescita.plot(c='r', linewidth=4, alpha=0.3)
            else:
                pred_crescita.plot(c='b', linewidth=3, alpha=0.3)

        for luogo, d in dati.groupby('luogo'):
            if not d['totale_casi'].iloc[-1] > min_casi:
                continue

            d = d.copy().set_index('data', drop=False)
            d['crescita'].where(d['crescita'] > 0).plot(
                xlim=xlim,
                ylim=ylim,
                alpha=0.2,
                linewidth=3,
        #         c='k',
            )

            x = d
            x = x[x['data'] > xlim[0]]
            #x = x[x['totale_casi'] > 300]
            x = x[ylim[0] < x['crescita']]
            x = x[x['crescita'] < ylim[1]]
            #x = x[(x['crescita'] == x['crescita'].max()) | (x['crescita'] == x['crescita'].min())]

            for y in x.iloc:
                ax.annotate(luogo, (y['data'].to_datetime64(), y['crescita']), c='k', va='center', ha='center', alpha=0.5)

        for y in italia.iloc[::5].iloc:
            ax.annotate('ITALIA', (y['data'].to_datetime64(), y['crescita']), c='r', va='bottom', ha='center', alpha=0.5, fontsize=14)

        if logy:
            logy_name = 'log'
        else:
            logy_name = 'lin'
        plt.savefig(f"{risoluzione_nome}-{logy_name}-{dati['data'].max().to_pydatetime().date().isoformat()}.png")

In [ ]:
dati_province.groupby('data')['totale_casi'].sum()[dati_province['data'].iloc[0]]

In [ ]:
xlim_date = [pd.to_datetime('2020-02-26'), pd.to_datetime('2020-03-31')]
xlim = [d.to_datetime64() for d in xlim_date]

for risoluzione_nome, dati_orig, scatter_size, min_casi in [
    ('province', dati_province, 0.2, 500),
    ('regioni', dati_regioni, 0.1, 100),
]:
    for logy in [True, False]:
        if logy:
            ylim = (0.01, 1)
        else:
            ylim = (-0.1, 1.5)

        dati = dati_orig.copy()
        
        dati[['totale_casi', 'crescita']] = dati.groupby('luogo').rolling(5, win_type='hamming', center=True, min_periods=1)[['totale_casi', 'crescita']].mean().reset_index('luogo', drop=True)
        
        dati.plot.scatter(
            'data', 
            'crescita', 
            s=0.2 * dati['totale_casi'],
            alpha=0.1,
            logy=logy,
            xlim=xlim,
            ylim=ylim, 
            figsize=(20, 15),
        )
        ax = plt.gca()

        for coeff, style in [
            [1.00, 'r'],
            [0.50, 'r'],
            [0.20, 'r'],
            [0.10, 'y'],
            [0.05, 'y'],
            [0.02, 'g'],
        ]:
            ax.plot(xlim, [coeff, coeff], style, linewidth=1, alpha=0.3)
            ax.annotate(f"+{coeff * 100:.0f}%", (xlim[1], coeff * 1.1), alpha=0.5, ha='right')

        dati[
            dati['totale_casi'] > 100
        ].set_index(['luogo', 'data'])['crescita'].unstack('luogo').plot(ax=ax, style='-', alpha=0.3, linewidth=0.5, legend=False)
            
        for luogo, d in dati.groupby('luogo'):
            if not d['totale_casi'].iloc[-1] > min_casi:
                continue

            d = d.copy().set_index('data', drop=False)
#             d['crescita'].where(d['crescita'] > 0).plot(
#                 xlim=xlim,
#                 ylim=ylim,
#                 alpha=0.2,
#                 linewidth=3,
#                 c='k',
#             )

            x = d
            x = x[x['data'] > xlim[0]]
            x = x[x['totale_casi'] > 2000]
            x = x[ylim[0] < x['crescita']]
            x = x[x['crescita'] < ylim[1]]
            #x = x[(x['crescita'] == x['crescita'].max()) | (x['crescita'] == x['crescita'].min())]

            for y in x.iloc:
                ax.annotate(luogo, (y['data'].to_datetime64(), y['crescita']), c='k', va='center', ha='center', alpha=0.5)
        plt.show()

In [ ]:
raise AssertionError
groups = list(dati_province[dati_province['data'] > '2020-03-10'].groupby('data'))
groups.reverse()
fig, axes = plt.subplots(len(groups), 1, figsize=(20, 20 * len(groups)), sharey=True)

ylim = [0.01, 2]

for (d, g), ax in zip(groups, axes):
    ax.set_title(f"{d}")

    data = g.set_index('denominazione_provincia')

    ax.set_yscale('log')
    ax.set_xscale('log')
    data.plot.scatter('totale_casi', 'crescita', marker='+', xlim=(1e1, 1e4), ylim=ylim, loglog=True, ax=ax)

    for casi in [1e2, 1e3]:
        ax.plot([casi, casi], ylim, 'k', linewidth=1, alpha=0.2)

    for coeff, style in [
        [1.00, 'r'],
        [0.50, 'r'],
        [0.20, 'r'],
        [0.10, 'y'],
        [0.05, 'y'],
        [0.02, 'g'],
    ]:
        ax.plot([1, 1e5], [coeff, coeff], style, linewidth=1, alpha=0.5)
        ax.annotate(f"+{coeff * 100:.0f}%", (1e4, coeff * 1.1), alpha=0.5, ha='right')

    for provincia in data.index:
        x, y = data.totale_casi[provincia], data.crescita[provincia]

        if x >= 20 and y > 0 and data.crescita[provincia] > 0.01:
            ax.annotate(
                f"{provincia}\n{int(data['stima_attualmente_positivi'][provincia])}/{data.totale_casi[provincia]} (+{data.crescita[provincia]*100:.1f}%)",
                (x, y * 1.05),
                alpha=0.4, rotation=0, rotation_mode='anchor', va='bottom', ha='center', ma='center',
            )

In [ ]:
xlim_date = [pd.to_datetime('2020-03-12'), pd.to_datetime('2020-03-28')]
xlim = [d.to_datetime64() for d in xlim_date]

for risoluzione_nome, dati, scatter_size, min_casi in [
    ('province', dati_province, 0.1, 2000),
#     ('regioni', dati_regioni, 0.05, 4000),
]:
    for logy in [True]:
        if logy:
            ylim = (0.08, 0.25)
        else:
            ylim = (-0.1, 1.5)

        italia = dati_andamento_nazionale

        italia.plot.scatter(
            'data', 
            'crescita', 
            s=0.05 * italia['totale_casi'], 
            color='r', 
            alpha=0,
            logy=logy, 
            ylim=ylim, 
            figsize=(5, 5),
        )
        ax = plt.gca()
        
        ax.set_xlim(xlim)

        italia.set_index('data')['crescita'].plot(style='o-r', alpha=0.5, linewidth=2)

        for coeff, style in [
            [1.00, 'r'],
            [0.50, 'r'],
            [0.20, 'r'],
            [0.10, 'y'],
            [0.05, 'y'],
            [0.02, 'g'],
        ]:
            ax.plot(xlim, [coeff, coeff], style, linewidth=1, alpha=0.3)
            ax.annotate(f"+{coeff * 100:.0f}%", (xlim[1], coeff * 1.1), alpha=0.5, ha='right')
